# ResNet34 or ResNet18 

[pretrained-resnet34-in-keras](https://www.kaggle.com/meaninglesslives/pretrained-resnet34-in-keras)

In [ ]:
import keras.backend as K
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Dense, Add
from keras.models import Model
from keras.engine import get_source_inputs
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


import keras
from distutils.version import StrictVersion

if StrictVersion(keras.__version__) < StrictVersion('2.2.0'):
    from keras.applications.imagenet_utils import _obtain_input_shape
else:
    from keras_applications.imagenet_utils import _obtain_input_shape

def build_resnet(
     repetitions=(2, 2, 2, 2),
     include_top=True,
     input_tensor=None,
     input_shape=None,
     classes=1000,
     block_type='usual'):

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      data_format='channels_last',
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape, name='data')
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # get parameters for model layers
    no_scale_bn_params = get_bn_params(scale=False)
    bn_params = get_bn_params()
    conv_params = get_conv_params()
    init_filters = 64

    if block_type == 'basic':
        conv_block = basic_conv_block
        identity_block = basic_identity_block
    else:
        conv_block = usual_conv_block
        identity_block = usual_identity_block
    
    # resnet bottom
    x = BatchNormalization(name='bn_data', **no_scale_bn_params)(img_input)
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(init_filters, (7, 7), strides=(2, 2), name='conv0', **conv_params)(x)
    x = BatchNormalization(name='bn0', **bn_params)(x)
    x = Activation('relu', name='relu0')(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pooling0')(x)
    
    # resnet body
    for stage, rep in enumerate(repetitions):
        for block in range(rep):
            
            filters = init_filters * (2**stage)
            
            # first block of first stage without strides because we have maxpooling before
            if block == 0 and stage == 0:
                x = conv_block(filters, stage, block, strides=(1, 1))(x)
                
            elif block == 0:
                x = conv_block(filters, stage, block, strides=(2, 2))(x)
                
            else:
                x = identity_block(filters, stage, block)(x)
                
    x = BatchNormalization(name='bn1', **bn_params)(x)
    x = Activation('relu', name='relu1')(x)

    # resnet top
    if include_top:
        x = GlobalAveragePooling2D(name='pool1')(x)
        x = Dense(classes, name='fc1')(x)
        x = Activation('softmax', name='softmax')(x)

    # Ensure that the model takes into account any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Create model.
    model = Model(inputs, x)

    return model


Using TensorFlow backend.


In [ ]:
def handle_block_names(stage, block):
    name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
    conv_name = name_base + 'conv'
    bn_name = name_base + 'bn'
    relu_name = name_base + 'relu'
    sc_name = name_base + 'sc'
    return conv_name, bn_name, relu_name, sc_name


def basic_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer


def basic_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        shortcut = Conv2D(filters, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        shortcut = Conv2D(filters*4, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer


In [ ]:
from keras.utils import get_file

def get_conv_params(**params):
    default_conv_params = {
        'kernel_initializer': 'glorot_uniform',
        'use_bias': False,
        'padding': 'valid',
    }
    default_conv_params.update(params)
    return default_conv_params

def get_bn_params(**params):
    default_bn_params = {
        'axis': 3,
        'momentum': 0.99,
        'epsilon': 2e-5,
        'center': True,
        'scale': True,
    }
    default_bn_params.update(params)
    return default_bn_params


def find_weights(weights_collection, model_name, dataset, include_top):
    w = list(filter(lambda x: x['model'] == model_name, weights_collection))
    w = list(filter(lambda x: x['dataset'] == dataset, w))
    w = list(filter(lambda x: x['include_top'] == include_top, w))
    return w


def load_model_weights(weights_collection, model, dataset, classes, include_top):
    weights = find_weights(weights_collection, model.name, dataset, include_top)

    if weights:
        weights = weights[0]

        if include_top and weights['classes'] != classes:
            raise ValueError('If using `weights` and `include_top`'
                             ' as true, `classes` should be {}'.format(weights['classes']))

        weights_path = get_file(weights['name'],
                                weights['url'],
                                cache_subdir='models',
                                md5_hash=weights['md5'])

        model.load_weights(weights_path)

    else:
        raise ValueError('There is no weights for such configuration: ' +
                         'model = {}, dataset = {}, '.format(model.name, dataset) +
                         'classes = {}, include_top = {}.'.format(classes, include_top))

In [ ]:
weights_collection = [
    # ResNet34
    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': True,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000.h5',
        'name': 'resnet34_imagenet_1000.h5',
        'md5': '2ac8277412f65e5d047f255bcbd10383',
    },

    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': False,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000_no_top.h5',
        'name': 'resnet34_imagenet_1000_no_top.h5',
        'md5': '8caaa0ad39d927cb8ba5385bf945d582',
    },
]


In [ ]:
def ResNet34(input_shape, input_tensor=None, weights=None, classes=1000, include_top=True):
    model = build_resnet(input_tensor=input_tensor,
                         input_shape=input_shape,
                         repetitions=(3, 4, 6, 3),
                         classes=classes,
                         include_top=False,
                         block_type='basic')
    model.name = 'resnet34'

    if weights:
        load_model_weights(weights_collection, model, weights, classes, include_top)
    return model

In [ ]:
base_model = ResNet34(input_shape=(101,101, 3), weights="imagenet", classes=100, include_top=False)

In [ ]:
kk = [layer for layer in base_model.layers]
len(kk)    

158

In [ ]:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.shape)

In [ ]:
base_model.get_layer(base_model.layers[70].name).output

<tf.Tensor 'zero_padding2d_84/Pad:0' shape=(?, 15, 15, 128) dtype=float32>

In [ ]:
# x = base_model.get_layer("stage2_unit4_conv2").output
x = base_model.get_layer(base_model.layers[83].name).output
# x = Conv2D(100, (2,2))(x)
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
# x = Dense(100, activation='relu')(x)
# and a logistic layer -- let's say we have 100 classes
predictions = Dense(100, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers[:71]:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=SGD(0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
# model.fit_generator(...)


In [ ]:
# base_model.get_layer(base_model.layers[74].name).output

In [ ]:
base_model.summary()

In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 101, 101, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 101, 101, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_69 (ZeroPadding2 (None, 107, 107, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 51, 51, 64)   9408        zero_padding2d_69[0][0]          
___________________________________________________________________________________________

In [ ]:
from keras.datasets import cifar100
import numpy as np
import cv2
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# x_train_mean = np.mean(x_train, axis=(0,1,2))
# x_train_std = np.std(x_train, axis=(0,1,2))

# x_test_mean = np.mean(x_test, axis=(0,1,2))
# x_test_std = np.std(x_test, axis=(0,1,2))

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')

# x_train = np.asarray([(x - x_train_mean) / x_train_std for x in x_train ])
# x_test = np.asarray([(x - x_test_mean) / x_test_std for x in x_test ])



# train_mean = np.mean(x_train, axis=(0,1,2))
# train_std = np.std(x_train, axis=(0,1,2))

# normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
# pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

from  skimage import transform
new_shape = (101,101)
print("strt")
x_train = np.asarray([cv2.resize(image, new_shape) for image in x_train])
x_test = np.asarray([transform.resize(image, new_shape) for image in x_test])

# x_train = normalize(pad4(x_train))
# x_test = normalize(pad4(x_test))

strt


In [ ]:
# import tensorflow as tf
# crop = lambda x : tf.image.random_flip_left_right(tf.random_crop(x, [50000,32, 32, 3]))
# x_train = crop(x_train)

In [ ]:
x_train[0].shape

(32, 32, 3)

In [ ]:
# x_train_mean = np.mean(x_train, axis=(0,1,2))
# x_train_std = np.std(x_train, axis=(0,1,2))
# print(x_train_mean, x_train_std)

In [ ]:
# x_test_mean = np.mean(x_test, axis=(0,1,2))
# x_test_std = np.std(x_test, axis=(0,1,2))
# print(x_test_mean, x_test_std)

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes=100, dtype='float32')
y_test = keras.utils.to_categorical(y_test, num_classes=100, dtype='float32')

In [ ]:
cutout_proba = 0.2
ssr_proba = 0.15
strong_aug_proba = 0.05

start_reg_ssr = 5
end_reg_ssr = 24

start_strong_aug = 3
end_strong_aug = 12

from albumentations import (
    Compose, HorizontalFlip, Rotate,GaussNoise ,RandomCrop,
    RandomBrightness, RandomContrast,OneOf,
    ToFloat, ShiftScaleRotate,PadIfNeeded
)

from albumentations import (HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
                            Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
                            IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
                            IAASharpen, IAAEmboss, Flip, OneOf, Compose, ChannelShuffle,RandomContrast, RandomCrop)

import cv2


def get_cutout_eraser(p=0.4, s_l=0.05, s_h=0.3, r_1=0.3, r_2=1/0.3, 
                      v_l=-1,v_h=1,
                      max_erasures_per_image=1, pixel_level=False):
    assert max_erasures_per_image >= 1

    def eraser(input_img):
#         print(input_img.shape)
        img_h, img_w, img_c = input_img.shape
        shape = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img
        mx = np.random.randint(1, max_erasures_per_image + 1)
        for i in range(mx):
            while True:
                s = np.random.uniform(s_l, s_h) * img_h * img_w
                r = np.random.uniform(r_1, r_2)
                w = int(np.sqrt(s / r))
                h = int(np.sqrt(s * r))
                left = np.random.randint(0, img_w)
                top = np.random.randint(0, img_h)

                if left + w <= img_w and top + h <= img_h:
                    break

            if pixel_level:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            else:
                c = np.mean(input_img, axis=(0,1))

            input_img[top:top + h, left:left + w, :] = c
        return input_img

    return eraser


# def multiple_aug(x, y):
#     x = tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3]))
#     return (x, y)

# data_aug = lambda x, y: multiple_aug(x, y)


cutout_fn = get_cutout_eraser(p=0.5, pixel_level=True)

def batch_cut(imgs):
    imgs = cutout_fn(imgs)
    return imgs


ssr = ShiftScaleRotate(p=ssr_proba)
rotate_one = lambda x: ssr(image=x)['image']

def rotate(imgs):
    for i,im in enumerate(imgs):
        imgs[i] = rotate_one(im)
    return imgs


AUGMENTATIONS_FOR_TRAIN = Compose([GaussNoise(p=0.25),
                      RandomContrast(limit=0.2, p=0.5),
                      HorizontalFlip(p=0.5),
                      RandomCrop(32,32,p=1.0),
                      PadIfNeeded(32,32),
                      RandomBrightness(limit=0.2, p=0.5),
                      ShiftScaleRotate(shift_limit=0.052, scale_limit=0.1, 
                                       rotate_limit=0, border_mode=cv2.BORDER_REFLECT_101, p=0.8)],
                     p=strong_aug_proba)


mix_aug = lambda x: AUGMENTATIONS_FOR_TRAIN(image=x)['image']
def other_augs(imgs):
    imgs = mix_aug(imgs)
    return imgs

#Apply AUG
def combined_aug(imgs):
    imgs = np.copy(imgs)
    imgs = rotate(imgs)
    imgs = batch_cut(imgs)
    # imgs = other_augs(imgs)
    return imgs

In [ ]:
# performing data argumentation by training image generator
dataAugmentaion = ImageDataGenerator(preprocessing_function = combined_aug)

# training the model
model.fit_generator(dataAugmentaion.flow(x_train, y_train, batch_size = 64),
                    validation_data = (x_test, y_test), 
                    steps_per_epoch = len(x_train) // 64,
                    epochs = 50,
                    ) 



# model.fit(x_train, y_train, 
#           batch_size=None, 
#           epochs=100, 
#           verbose=1, 
#           callbacks=None, 
# #           validation_split=0.0, 
#           validation_data=(x_test, y_test), 
#           shuffle=True, 
#           use_multiprocessing=True)

Epoch 1/50
781/781 [==============================] - 44s 56ms/step - loss: 4.1246 - acc: 0.0942 - val_loss: 4.8398 - val_acc: 0.0100
Epoch 2/50
781/781 [==============================] - 41s 53ms/step - loss: 2.9772 - acc: 0.2679 - val_loss: 5.2011 - val_acc: 0.0100
Epoch 3/50
781/781 [==============================] - 42s 54ms/step - loss: 2.4448 - acc: 0.3648 - val_loss: 6.0034 - val_acc: 0.0097
Epoch 4/50
781/781 [==============================] - 41s 53ms/step - loss: 2.1741 - acc: 0.4228 - val_loss: 6.4674 - val_acc: 0.0100
Epoch 5/50
638/781 [=======================>......] - ETA: 6s - loss: 2.0091 - acc: 0.4595

In [ ]:
dataAugmentaion = ImageDataGenerator(preprocessing_function = combined_aug)

# training the model
model.fit_generator(dataAugmentaion.flow(x_train, y_train, batch_size = 16),
                    validation_data = (x_test, y_test), 
                    steps_per_epoch = len(x_train) // 32,
                    epochs = 20,
                    ) 

In [ ]:
dataAugmentaion = ImageDataGenerator(preprocessing_function = combined_aug)

# training the model
model.fit_generator(dataAugmentaion.flow(x_train, y_train, batch_size = 32),
                    validation_data = (x_test, y_test), 
                    steps_per_epoch = len(x_train) // 32,
                    epochs = 40,
                    ) 

In [ ]:
kk = model.evaluate(x_test, y_test)
# scores = model.evaluate(X[test], Y[test], verbose=0)


In [ ]:
print(kk)

In [ ]:
base_model.layers[:155]